## ライブラリのインポート

In [1]:
from keras.models import Sequential
from keras.utils import np_utils, plot_model
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.initializers import glorot_uniform, orthogonal, TruncatedNormal
from keras.callbacks import EarlyStopping
from keras.layers.recurrent import GRU, SimpleRNN

import pandas as pd #行列計算
import numpy as np #行列計算
import math #数値計算
import itertools #順列・組み合わせ
import time

import matplotlib.pyplot as plt #グラフ
import winsound # ビープ音
import gc

from df_method import rise_fall_rate, moving_average, GCDC, df_shift, add_data, RSI, Z_score_normalization, Min_Max_normalization

Using TensorFlow backend.


## モデル構築

In [2]:
class Prediction :
    #初期化
    def __init__(self, maxlen, n_hidden, n_in, n_out, learning_model):
        self.maxlen = maxlen #入力系列数
        self.n_hidden = n_hidden #出力次元（隠れ層内のニューロン数）
        self.n_in = n_in #学習データの列数
        self.n_out = n_out #ラベルデータの列数
        
        self.learning_model = learning_model #●学習モデルの選択

    #モデルの生成
    def create_model(self):
        model = Sequential()
        if self.learning_model == 'RNN':
            #RNN層
            model.add(SimpleRNN(self.n_hidden,
                                batch_input_shape = (None, self.maxlen, self.n_in),
                                kernel_initializer = glorot_uniform(seed=20170719),
                                recurrent_initializer = orthogonal(gain=1.0, seed=20170719),
                                dropout = 0.5,
                                recurrent_dropout = 0.5))
        elif self.learning_model == 'LSTM':
            #LSTM層
            model.add(LSTM(self.n_hidden,
                           batch_input_shape = (None, self.maxlen, self.n_in),
                           kernel_initializer = glorot_uniform(seed=20170719), 
                           recurrent_initializer = orthogonal(gain=1.0, seed=20170719), 
                           dropout = 0.5, 
                           recurrent_dropout = 0.5))
        elif self.learning_model == 'GRU':
            #GRU層
            model.add(GRU(self.n_hidden,
                          batch_input_shape = (None, self.maxlen, self.n_in),
                          kernel_initializer = glorot_uniform(seed=20170719),
                          recurrent_initializer = orthogonal(gain=1.0, seed=20170719),
                          dropout = 0.5,
                          recurrent_dropout = 0.5))
        #ドロップアウト層
        model.add(Dropout(0.5))
        #結合層
        model.add(Dense(self.n_out, kernel_initializer = glorot_uniform(seed=20170719)))
        #活性化層
        model.add(Activation("softmax"))
        #コンパイル
        model.compile(loss="categorical_crossentropy", optimizer = "Adam", metrics = ['categorical_accuracy']) # "RMSprop"
        return model

    # 学習
    def train(self, x_train, t_train, batch_size, epochs) :
        early_stopping = EarlyStopping(patience=0, verbose=1)
        model = self.create_model()
        self.hist = model.fit(x_train, t_train, batch_size = batch_size, epochs = epochs, verbose = 1,
                              shuffle = True, callbacks = [early_stopping], validation_split = 0.1)
        return model

## 変数宣言

In [3]:
is_debug = True
csv_path = './csv_realtime/'
#day_list = ['1'] # '1', '2', '7', '30', '365'
x_days_later = 1
learning_model_list = ['LSTM']#['RNN', 'LSTM', 'GRU']
year_list = ['2009-3-10']#  '1960', '1970', '1980', '1990', '2000', '2010'
end_date = '2018-10-31'# 終点年月日

min_maxlen = 100
max_maxlen = 1000#1000
min_n_hidden = 100
max_n_hidden = 500#500

target_name = 'USD_JPY_diff'# 'nikkei_Close', 'nikkei_diff', 'USD_JPY'

max_correct = 0 # 最高正答率
max_correct_number = -1 # 最高正答率を出したデータフレームの番号 

## 絶対に必要なデータフレームを用意

In [4]:
df_target = add_data(csv_path+'USD_JPY.csv', x_days_later)

#ドル円の始値と終値の差
usd_jpy_diff = pd.read_csv(csv_path+'USD_JPY.csv', index_col='Date', parse_dates=True)#読み込み
usd_jpy_diff = usd_jpy_diff.apply(np.log)*100#正規化
usd_jpy_diff = usd_jpy_diff['USD_JPY_Close'] - usd_jpy_diff['USD_JPY_Open']#終値と始値の差を求める
usd_jpy_diff = usd_jpy_diff['1989-10-16':]#始値と終値と高値と安値が記録され始めた日からのみ抽出
#usd_jpy_diff = usd_jpy_diff.diff(x_days_later)#特定の日数後の増減を求める
#usd_jpy_diff = usd_jpy_diff.drop(df.index[0:x_days_later], axis=0)#特定の日数分の行を削除
#usd_jpy_diff = df_shift(df, 1)
usd_jpy_diff = usd_jpy_diff.rename('USD_JPY_diff')#名前を付ける
print(usd_jpy_diff)

df_target = df_target.join(usd_jpy_diff, how='inner')
print(df_target)

Date
1989-10-16    0.105734
1989-10-17    0.028052
1989-10-18    0.042400
1989-10-19    0.077582
1989-10-20    0.049142
1989-10-23    0.007045
1989-10-24    0.000000
1989-10-25    0.007066
1989-10-26    0.035217
1989-10-27    0.035312
1989-10-30    0.035156
1989-10-31    0.035008
1989-11-01    0.034794
1989-11-02    0.034874
1989-11-03    0.020931
1989-11-06    0.020856
1989-11-07    0.027996
1989-11-08    0.034898
1989-11-09    0.013980
1989-11-10    0.034861
1989-11-13    0.027807
1989-11-14    0.034947
1989-11-15    0.013907
1989-11-16    0.013878
1989-11-17    0.034644
1989-11-20    0.020782
1989-11-21    0.006966
1989-11-22    0.034930
1989-11-23    0.034692
1989-11-24    0.034825
                ...   
2018-10-01    0.237206
2018-10-02   -0.246024
2018-10-03    0.753861
2018-10-04   -0.472938
2018-10-05   -0.140622
2018-10-08   -0.414262
2018-10-09   -0.221112
2018-10-10   -0.612655
2018-10-11   -0.106952
2018-10-12    0.071339
2018-10-15   -0.366186
2018-10-16    0.419662
2018-1

## 検証するデータフレームを用意

In [5]:
df_x_name = 'USD_JPY'

df_list_name = [
    'NASDAQ',
    'EUR_JPY',
    'EUR_USD',
    'nikkei',
    'DOW30',
    'treasury_10',
]

df_list = [] #データフレームのリスト

df_list.append(add_data(csv_path+'NASDAQ.csv', x_days_later))
df_list.append(add_data(csv_path+'EUR_JPY.csv', x_days_later))
df_list.append(add_data(csv_path+'EUR_USD.csv', x_days_later))
df_list.append(add_data(csv_path+'nikkei.csv', x_days_later))
df_list.append(add_data(csv_path+'DOW30.csv', x_days_later))
            
#f_list.append(add_data(csv_path+'TNX.csv'))#1米国債10年？
#_list.append(add_data(csv_path+'GSPC.csv'))#S&P500
#df_list.append(add_data(csv_path+'RUT.csv'))#Russell2000
#df_list.append(add_data(csv_path+'TOPIX.csv'))
#df_list.append(add_data(csv_path+'BTC_USD.csv'))
                        
# 米国債
df = pd.read_csv(csv_path+'treasury_10.csv', index_col='Date', parse_dates=True)
df_list.append(rise_fall_rate(df, x_days_later))

print(df_list)

[            NASDAQ_Open  NASDAQ_High  NASDAQ_Low  NASDAQ_Close  \
Date                                                             
1971-02-08     0.836488     0.836488    0.836488      0.836488   
1971-02-09    -0.079359    -0.079359   -0.079359     -0.079359   
1971-02-10    -0.069496    -0.069496   -0.069496     -0.069496   
1971-02-11     0.751953     0.751953    0.751953      0.751953   
1971-02-12     0.589688     0.589688    0.589688      0.589688   
1971-02-16     0.137093     0.137093    0.137093      0.137093   
1971-02-17    -0.441333    -0.441333   -0.441333     -0.441333   
1971-02-18    -0.315023    -0.315023   -0.315023     -0.315023   
1971-02-19    -0.712452    -0.712452   -0.712452     -0.712452   
1971-02-22    -1.018071    -1.018071   -1.018071     -1.018071   
1971-02-23     0.040121     0.040121    0.040121      0.040121   
1971-02-24     0.918351     0.918351    0.918351      0.918351   
1971-02-25     0.584540     0.584540    0.584540      0.584540   
1971-02-2

## メイン処理

In [ ]:
result_csv = pd.DataFrame(columns=['time', 'day', 'model', 'start date', 'end date', 'maxlen', 'n_hidden', 'correct', 'semi_correct', 'combination'])
debug_csv = pd.DataFrame(columns=['time', 'x_train.shape', 't_train.shape', 'batch size', 'epochs'])

# 学習モデル
for learning_model in learning_model_list:
    # 学習期間
    for year in year_list:
        while True:
            # 学習データ
            max_correct_number = -1 # 最高正答率を出したデータフレームの番号 
            for df, df_name, df_number in zip(df_list, df_list_name, range(len(df_list))):
                # 基本となる入力をdf_xに代入
                df_x = df_target
            
                # データフレームを一つ加える
                df_x = df_x.join(df, how='inner', rsuffix='_' + df_name)         
            
                #指定の期間を抽出
                if end_date == '':
                    df_x = df_x[year:]
                else:
                    df_x = df_x[year:end_date]
                
                # ●デバッグ
                if is_debug == True:
                    print(df_x)
            
                # ラベルデータを作成する列を抽出------------------------------------------------------
                target = df_x[target_name]
            
                #空のデータフレーム作成
                df_t = pd.DataFrame(index=df_x.index, columns=['jump', 'rise', 'fall', 'drop'])
                df_t = df_t.fillna(0) #０で埋める

                #条件にあった値を置換する
                df_t.loc[0.995033085 <= target, 'jump'] = 1
                df_t.loc[(0 <= target) & (target < 0.995033085), 'rise'] = 1
                df_t.loc[(-0.995033085 <= target) & (target < 0), 'fall'] = 1
                df_t.loc[target < -0.995033085, 'drop'] = 1

                df_t = df_t.shift(-1 * x_days_later, axis=0)#予測先日数分だけ縦にずらす
                df_t = df_t.drop(df_t.index[-1*x_days_later:], axis=0)#ラベルデータ末尾の行を削除
                df_x = df_x.drop(df_x.index[-1*x_days_later:], axis=0)#学習データの末尾の行を削除

                # ●デバッグ
                if is_debug == True:
                    print('df_t.shape=', df_t.shape, '\n')
            
                #インデックスと列名を外し２次元配列に変換------------------------------------
                x_data = df_x.values
                t_data = df_t.values
        
                if is_debug == True:
                    print('x_data.shape =', x_data.shape)
                    print('t_data.shape =', t_data.shape, '\n')
            
                # 入力系列------------------------------------------------------
                maxlen = min_maxlen
                while maxlen <= max_maxlen:
                    n_in = x_data.shape[1]   # 学習データ（＝入力）の列数
                    n_out = t_data.shape[1]  # ラベルデータ（=出力）の列数
                    len_seq = x_data.shape[0] - maxlen + 1
                    print('len_seq', len_seq, '\n')#●デバッグ
                    data = []
                    target = []

                    #
                    for i in range(0, len_seq):
                        data.append(x_data[i:i+maxlen, :])
                        target.append(t_data[i+maxlen-1, :])

                    x = np.array(data).reshape(len(data), maxlen, n_in)
                    t = np.array(target).reshape(len(data), n_out)

                    if is_debug == True:
                        print('x.shape=', x.shape)
                        print('t.shape=', t.shape, '\n')

                    # ここからソースコードの後半
                    n_train = int(len(data)*0.9)              # 訓練データ長
                    x_train,x_test = np.vsplit(x, [n_train])  # 学習データを訓練用とテスト用に分割
                    t_train,t_test = np.vsplit(t, [n_train])  # ラベルデータを訓練用とテスト用に分割

                    if is_debug == True:
                        print('x_train.shape=', x_train.shape)
                        print('x_test.shape=', x_test.shape, '\n')
                        print('t_train.shape=', t_train.shape)
                        print('t_test.shape=', t_test.shape, '\n')
                   
                    # メイン処理----------------------------------------------------------
                    # 隠れ層のユニット
                    n_hidden = min_n_hidden
                    while n_hidden <= max_n_hidden:
                        epochs = 100      # エポック数（同じデータでの学習回数）
                        batch_size = 256  #バッチサイズ
                
                        #パラメータの表示
                        print('model: ', learning_model)
                        print('day: ', x_days_later)
                        print('since: ', year)
                        print('maxlen: ', maxlen)
                        print('n_hidden: ', n_hidden, '\n')
                
                        # モデル定義
                        prediction = Prediction(maxlen, n_hidden, n_in, n_out, learning_model)
                    
                        # 学習時間の計測開始
                        start = time.time()
                    
                        # 学習（●学習時間がどんどん伸びていくのはなぜなのか？）
                        model = prediction.train(x_train, t_train, batch_size, epochs)
                    
                        # 学習時間の計測終了と表示
                        end = time.time()

                        #予測精度の評価------------------------------------------------------

                        # 正答率、準正答率（騰落）集計
                        preds = model.predict(x_test)
                
                        #正解数を数える変数
                        correct = 0
                        semi_correct = 0
                
                        #表を作るためのデータフレーム
                        matrix = pd.DataFrame(columns=['jump!', 'rise!', 'fall!', 'drop!'], index=['jump?', 'rise?', 'fall?', 'drop?'])
                        matrix = matrix.fillna(0)

                        #正解数を数える
                        for i in range(len(preds)):
                            pred = np.argmax(preds[i,:])#argmaxとは配列の最大要素のインデックスを返すメソッドである
                            tar = np.argmax(t_test[i,:])
                            matrix.iat[pred, tar] = matrix.iat[pred, tar] + 1 #●マトリックスのセルをインクリメント
                            if pred == tar :#完全一致
                               correct += 1
                            else :
                                if pred+tar == 1 or pred+tar == 5 :
                                    semi_correct += 1
                
                        #正答率と準正答率を求める
                        correct_rate = 1.0 *correct / len(preds) 
                        semi_correct_rate = 1.0 * (correct+semi_correct) / len(preds)
                    
                        # もし正答率が最高記録をだしたら更新
                        if semi_correct_rate > max_correct:
                            max_correct = semi_correct_rate
                            max_correct_number = df_number
                
                        #csvに記録
                        series = pd.Series([int(end - start),
                                            x_days_later,
                                            learning_model,
                                            year,
                                            end_date,
                                            maxlen,
                                            n_hidden,
                                            correct_rate,
                                            semi_correct_rate,
                                            df_x_name + ' ' + df_name],
                                            index = result_csv.columns)
                        result_csv = result_csv.append(series, ignore_index = True)
                        result_csv.to_csv('./log/log.csv', index=False)
                        
                        #csvに記録
                        series2 = pd.Series([int(end - start),
                                             x_train.shape,
                                             t_train.shape,
                                             batch_size,
                                             epochs],
                                             index = debug_csv.columns)
                        debug_csv = debug_csv.append(series2, ignore_index = True)
                        debug_csv.to_csv('./log/debug.csv', index=False)
                
                        #表と正答率と学習時間を表示
                        print('\n', matrix, '\n')
                        print("正答率:", 1.0 * correct / len(preds))
                        print("準正答率（騰落）:", 1.0 * (correct+semi_correct) / len(preds))
                        print ("学習時間:{0}".format(end - start) + "[sec]\n")
                
                
                        #次のステップへ
                        #del prediction # 効果なし
                        #gc.collect() # 効果なし
                        n_hidden += 100
                    
                    #次のステップへ
                    maxlen += 100
            
            # もし正答率を更新しなければ
            if max_correct_number == -1:
                break
            else:# そうでなければ新たなデータフレームをdf_targetに付属させてもう一回
                df_target = df_target.join(df_list.pop(max_correct_number))
                df_x_name += ' ' + df_list_name.pop(max_correct_number)

            USD_JPY_Open  USD_JPY_High  USD_JPY_Low  USD_JPY_Close  \
Date                                                                 
2009-03-10      0.578359     -0.040339     0.010213      -0.182279   
2009-03-11     -0.172143     -0.303061    -0.882147      -1.408618   
2009-03-12     -1.418753     -0.324281    -1.442457       0.451283   
2009-03-13      0.451283      0.131866     1.524847       0.326931   
2009-03-16      0.377918      0.000000     0.861014       0.142697   
2009-03-17      0.091710      0.323854     0.112205       0.416731   
2009-03-18      0.416731     -0.131439    -2.477162      -2.433260   
2009-03-19     -2.402086     -2.282013    -2.261775      -1.761421   
2009-03-20     -1.834914     -0.352551     0.649941       1.438723   
2009-03-23      1.251397      1.105326     1.580478       1.068198   
2009-03-24      1.308158      1.235403     1.318883       0.954784   
2009-03-25      0.975117     -0.203149     0.010316      -0.348004   
2009-03-26     -0.37

Train on 1887 samples, validate on 210 samples
Epoch 1/100
1887/1887 [==============================] - 3s 2ms/step - loss: 1.3770 - categorical_accuracy: 0.2819 - val_loss: 1.3145 - val_categorical_accuracy: 0.4286
Epoch 2/100
1887/1887 [==============================] - 1s 653us/step - loss: 1.2825 - categorical_accuracy: 0.4128 - val_loss: 1.0972 - val_categorical_accuracy: 0.5000
Epoch 3/100
1887/1887 [==============================] - 1s 614us/step - loss: 1.1084 - categorical_accuracy: 0.4626 - val_loss: 0.8595 - val_categorical_accuracy: 0.4667
Epoch 4/100
1887/1887 [==============================] - 1s 621us/step - loss: 1.0598 - categorical_accuracy: 0.4489 - val_loss: 0.8667 - val_categorical_accuracy: 0.4571
Epoch 00004: early stopping

        jump!  rise!  fall!  drop!
jump?      0      0      0      0
rise?      2    118    106      1
fall?      0      3      4      0
drop?      0      0      0      0 

正答率: 0.5213675213675214
準正答率（騰落）: 0.5299145299145299
学習時間:8.097653865

1806/1806 [==============================] - 2s 1ms/step - loss: 1.0449 - categorical_accuracy: 0.4607 - val_loss: 0.8543 - val_categorical_accuracy: 0.4975
Epoch 4/100
1806/1806 [==============================] - 2s 1ms/step - loss: 1.0128 - categorical_accuracy: 0.4723 - val_loss: 0.8468 - val_categorical_accuracy: 0.4826
Epoch 5/100
1806/1806 [==============================] - 2s 1ms/step - loss: 1.0073 - categorical_accuracy: 0.4790 - val_loss: 0.8654 - val_categorical_accuracy: 0.4826
Epoch 00005: early stopping

        jump!  rise!  fall!  drop!
jump?      0      0      0      0
rise?      1     21     24      0
fall?      1     93     83      1
drop?      0      0      0      0 

正答率: 0.4642857142857143
準正答率（騰落）: 0.4732142857142857
学習時間:14.788713693618774[sec]

model:  LSTM
day:  1
since:  2009-3-10
maxlen:  200
n_hidden:  400 

Train on 1806 samples, validate on 201 samples
Epoch 1/100
1806/1806 [==============================] - 4s 2ms/step - loss: 1.3175 - categorical_accura

1644/1644 [==============================] - 7s 4ms/step - loss: 1.3834 - categorical_accuracy: 0.2786 - val_loss: 1.3218 - val_categorical_accuracy: 0.4317
Epoch 2/100
1644/1644 [==============================] - 4s 2ms/step - loss: 1.3046 - categorical_accuracy: 0.4136 - val_loss: 1.1890 - val_categorical_accuracy: 0.4973
Epoch 3/100
1644/1644 [==============================] - 4s 3ms/step - loss: 1.1529 - categorical_accuracy: 0.4532 - val_loss: 0.8401 - val_categorical_accuracy: 0.4754
Epoch 4/100
1644/1644 [==============================] - 4s 3ms/step - loss: 1.0902 - categorical_accuracy: 0.4392 - val_loss: 0.8309 - val_categorical_accuracy: 0.4973
Epoch 5/100
1644/1644 [==============================] - 4s 2ms/step - loss: 1.0478 - categorical_accuracy: 0.4672 - val_loss: 0.8472 - val_categorical_accuracy: 0.4754
Epoch 00005: early stopping

        jump!  rise!  fall!  drop!
jump?      0      0      0      0
rise?      0     24     39      0
fall?      2     80     59      0
d

Train on 1563 samples, validate on 174 samples
Epoch 1/100
1563/1563 [==============================] - 9s 6ms/step - loss: 1.3424 - categorical_accuracy: 0.3442 - val_loss: 0.9908 - val_categorical_accuracy: 0.4943
Epoch 2/100
1563/1563 [==============================] - 5s 3ms/step - loss: 1.0989 - categorical_accuracy: 0.4651 - val_loss: 0.8592 - val_categorical_accuracy: 0.4885
Epoch 3/100
1563/1563 [==============================] - 5s 3ms/step - loss: 1.0310 - categorical_accuracy: 0.4530 - val_loss: 0.8055 - val_categorical_accuracy: 0.5057
Epoch 4/100
1563/1563 [==============================] - 5s 3ms/step - loss: 1.0191 - categorical_accuracy: 0.4383 - val_loss: 0.8388 - val_categorical_accuracy: 0.5000
Epoch 00004: early stopping

        jump!  rise!  fall!  drop!
jump?      0      0      0      0
rise?      1     20     31      0
fall?      1     80     61      0
drop?      0      0      0      0 

正答率: 0.4175257731958763
準正答率（騰落）: 0.422680412371134
学習時間:26.45026469230652[

...

1158/1158 [==============================] - 7s 6ms/step - loss: 1.2860 - categorical_accuracy: 0.4223 - val_loss: 1.0440 - val_categorical_accuracy: 0.4341
Epoch 3/100
1158/1158 [==============================] - 7s 6ms/step - loss: 1.1124 - categorical_accuracy: 0.4853 - val_loss: 0.8677 - val_categorical_accuracy: 0.4884
Epoch 4/100
1158/1158 [==============================] - 7s 6ms/step - loss: 1.1080 - categorical_accuracy: 0.4655 - val_loss: 0.8604 - val_categorical_accuracy: 0.4264
Epoch 5/100
1158/1158 [==============================] - 7s 6ms/step - loss: 1.0545 - categorical_accuracy: 0.4560 - val_loss: 0.9012 - val_categorical_accuracy: 0.4574
Epoch 00005: early stopping

        jump!  rise!  fall!  drop!
jump?      0      0      0      0
rise?      0     22     13      0
fall?      1     55     53      0
drop?      0      0      0      0 

正答率: 0.5208333333333334
準正答率（騰落）: 0.5208333333333334
学習時間:44.110305070877075[sec]

model:  LSTM
day:  1
since:  2009-3-10
maxlen:  100

Train on 1951 samples, validate on 217 samples
Epoch 1/100
1951/1951 [==============================] - 9s 5ms/step - loss: 1.3749 - categorical_accuracy: 0.3234 - val_loss: 1.3368 - val_categorical_accuracy: 0.4562
Epoch 2/100
1951/1951 [==============================] - 1s 707us/step - loss: 1.3000 - categorical_accuracy: 0.4331 - val_loss: 1.1668 - val_categorical_accuracy: 0.5023
Epoch 3/100
1951/1951 [==============================] - 1s 647us/step - loss: 1.1063 - categorical_accuracy: 0.4577 - val_loss: 0.8582 - val_categorical_accuracy: 0.4608
Epoch 4/100
1951/1951 [==============================] - 1s 708us/step - loss: 1.0678 - categorical_accuracy: 0.4557 - val_loss: 0.8690 - val_categorical_accuracy: 0.4470
Epoch 00004: early stopping

        jump!  rise!  fall!  drop!
jump?      0      0      0      0
rise?      2    105     95      1
fall?      0     19     19      0
drop?      0      0      0      0 

正答率: 0.5145228215767634
準正答率（騰落）: 0.5228215767634855
学習時間:14.31619787

1870/1870 [==============================] - 12s 6ms/step - loss: 1.3416 - categorical_accuracy: 0.3781 - val_loss: 0.8837 - val_categorical_accuracy: 0.4952
Epoch 2/100
1870/1870 [==============================] - 3s 1ms/step - loss: 1.0857 - categorical_accuracy: 0.4642 - val_loss: 0.9736 - val_categorical_accuracy: 0.4904
Epoch 00002: early stopping

        jump!  rise!  fall!  drop!
jump?      0      0      0      0
rise?      0      0      0      0
fall?      2    117    111      1
drop?      0      0      0      0 

正答率: 0.4805194805194805
準正答率（騰落）: 0.48484848484848486
学習時間:15.534166097640991[sec]

model:  LSTM
day:  1
since:  2009-3-10
maxlen:  200
n_hidden:  500 

Train on 1870 samples, validate on 208 samples
Epoch 1/100
1870/1870 [==============================] - 12s 6ms/step - loss: 1.3256 - categorical_accuracy: 0.3877 - val_loss: 0.9352 - val_categorical_accuracy: 0.4904
Epoch 2/100
1870/1870 [==============================] - 3s 1ms/step - loss: 1.0675 - categorical_acc

1708/1708 [==============================] - 15s 9ms/step - loss: 1.3681 - categorical_accuracy: 0.3372 - val_loss: 1.3142 - val_categorical_accuracy: 0.4684
Epoch 2/100
1708/1708 [==============================] - 4s 2ms/step - loss: 1.2049 - categorical_accuracy: 0.4473 - val_loss: 0.8211 - val_categorical_accuracy: 0.4789
Epoch 3/100
1708/1708 [==============================] - 4s 2ms/step - loss: 1.0480 - categorical_accuracy: 0.4678 - val_loss: 0.8346 - val_categorical_accuracy: 0.5000
Epoch 00003: early stopping

        jump!  rise!  fall!  drop!
jump?      0      0      0      0
rise?      2    106    102      1
fall?      0      0      0      0
drop?      0      0      0      0 

正答率: 0.5023696682464455
準正答率（騰落）: 0.5118483412322274
学習時間:24.139015436172485[sec]

model:  LSTM
day:  1
since:  2009-3-10
maxlen:  400
n_hidden:  300 

Train on 1708 samples, validate on 190 samples
Epoch 1/100
1708/1708 [==============================] - 15s 9ms/step - loss: 1.3680 - categorical_accu

Train on 1627 samples, validate on 181 samples
Epoch 1/100
1627/1627 [==============================] - 19s 11ms/step - loss: 1.3462 - categorical_accuracy: 0.3860 - val_loss: 0.7957 - val_categorical_accuracy: 0.5359
Epoch 2/100
1627/1627 [==============================] - 6s 3ms/step - loss: 1.0722 - categorical_accuracy: 0.4591 - val_loss: 0.8435 - val_categorical_accuracy: 0.5138
Epoch 00002: early stopping

        jump!  rise!  fall!  drop!
jump?      0      0      0      0
rise?      0      3      0      0
fall?      2    100     96      0
drop?      0      0      0      0 

正答率: 0.4925373134328358
準正答率（騰落）: 0.4925373134328358
学習時間:25.42298674583435[sec]

len_seq 1909 

x.shape= (1909, 600, 9)
t.shape= (1909, 4) 

x_train.shape= (1718, 600, 9)
x_test.shape= (191, 600, 9) 

t_train.shape= (1718, 4)
t_test.shape= (191, 4) 

model:  LSTM
day:  1
since:  2009-3-10
maxlen:  600
n_hidden:  100 

Train on 1546 samples, validate on 172 samples
Epoch 1/100
1546/1546 [====================

## 計算終了の合図

In [ ]:
for i in range(3):
    winsound.Beep(784,300)
    winsound.Beep(698,300)
    winsound.Beep(784,600)